# Check out the Clay v1 model on the dataset

In [3]:
import math

import geopandas as gpd
import numpy as np
import pandas as pd
import pystac_client
import stackstac
import torch
import yaml
from box import Box
from matplotlib import pyplot as plt
from rasterio.enums import Resampling
from shapely import Point
from torchvision.transforms import v2

import shapely
import json

from src.models.clay.model import ClayMAEModule

from src.data.get_satellite_images import ReadSTAC

In [4]:
api_url="https://planetarycomputer.microsoft.com/api/stac/v1"
bands = ['B04', 'B03', 'B02']

stac_reader = ReadSTAC(api_url=api_url)

# Load Dataset

In [5]:
data = gpd.read_file("/workspaces/mine-segmentation/data/raw/mining_tiles_with_masks.gpkg")

In [6]:
# select first row
row = data.iloc[0]
row

tile_id                                                         621
tile_bbox         {"type": "Polygon", "coordinates": [[[-63.3333...
sentinel_2_id     S2B_MSIL2A_20190517T144739_R139_T20NMP_2020100...
source_dataset                                                 tang
timestamp                                2024-06-19 09:07:50.341000
geometry          MULTIPOLYGON (((-63.417205999535305 7.45592197...
Name: 0, dtype: object

In [7]:
# convert geojson string to shapely geometry
geom = shapely.geometry.shape(json.loads(row.tile_bbox))
bounds = geom.bounds

In [8]:
item = stac_reader.get_item_by_name(row.sentinel_2_id, bbox=bounds)
item

<Item id=S2B_MSIL2A_20190517T144739_R139_T20NMP_20201006T152830>

In [9]:
stack = stac_reader.get_stack(item, bands)
stack

Loading stack...
Returning stack from single S2 image with ID: S2B_MSIL2A_20190517T144739_R139_T20NMP_20201006T152830


<xarray.DataArray 'stackstac-11c71b664883afe391fe4a90cf2dc9a6' (band: 3,
                                                                y: 1842, x: 1842)> Size: 81MB
dask.array<mosaic-aggregate, shape=(3, 1842, 1842), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/45)
  * band                                     (band) <U3 36B 'B04' 'B03' 'B02'
  * x                                        (x) float64 15kB 4.448e+05 ... 4...
  * y                                        (y) float64 15kB 8.29e+05 ... 8....
    s2:processing_baseline                   <U5 20B '02.12'
    s2:generation_time                       <U24 96B '2020-10-06T15:28:30.551Z'
    s2:nodata_pixel_percentage               float64 8B 0.0
    ...                                       ...
    proj:bbox                                object 8B {399960.0, 790200.0, 5...
    common_name                              (band) <U5 60B 'red' 'green' 'blue'
    center_wavelength                        (band) float64 24B 0.665 0.56 0.49
    full_width_half_max                      (band) float64 24B 0.038 ... 0.098
    epsg                                     int64 8B 32620
    s2_tile_id                               <U54 216B 'S2B_MSIL2A_20190517T1...
Attributes:
    spec:        RasterSpec(epsg=32620, bounds=(444810, 810620, 463230, 82904...
    crs:         epsg:32620
    transform:   | 10.00, 0.00, 444810.00|\n| 0.00,-10.00, 829040.00|\n| 0.00...
    resolution:  10
    s2_tile_id:  S2B_MSIL2A_20190517T144739_R139_T20NMP_20201006T152830

### Load the model

We now have the data to analyse, let's load the model.

In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
ckpt = "https://clay-model-ckpt.s3.amazonaws.com/v0.5.7/mae_v0.5.7_epoch-13_val-loss-0.3098.ckpt"
torch.set_default_device(device)

model = ClayMAEModule.load_from_checkpoint(
    ckpt, metadata_path="configs/metadata.yaml", shuffle=False, mask_ratio=0
)
model.eval()

model = model.to(device)

Downloading: "https://clay-model-ckpt.s3.amazonaws.com/v0.5.7/mae_v0.5.7_epoch-13_val-loss-0.3098.ckpt" to /root/.cache/torch/hub/checkpoints/mae_v0.5.7_epoch-13_val-loss-0.3098.ckpt
 13%|█▎        | 218M/1.61G [00:59<06:29, 3.85MB/s] 


KeyboardInterrupt: 